# This script uses some ODBC connection to execute select statements and pull data from database and write it into a Google Spreadsheet using google drive API

There are some execution control files and logs to ensure process and data consistency

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
import datetime
import sys
import os
import pypyodbc as pyodbc
import logging

def ending_log(start_time, error = False):
    if error:
        finishing_msg_01 = '[INFO] Finishing with errors...'
        finishing_msg_02 = '[INFO] Execution FAILED!'
    else:
        finishing_msg_01 = '[INFO] Finishing without errors..'
        finishing_msg_02 = '[INFO] Execution SUCCEEDED!'
    end_time_dt = datetime.datetime.today()
    execution_time_dt = end_time_dt - start_time
    execution_time_str = execution_time_dt.__str__()
    logging.info('[INFO] Execution time: ' + execution_time_str)
    logging.info(finishing_msg_01)
    logging.info(finishing_msg_02)
    logging.info('-----------------------------------------------------------')

ts = time.time()
today_str = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d')

#--------------------------- Start - Setting variables --------------------------#
# Path to execution control date file - this file contains the last date loaded into spreadsheet
execution_control_file_name = 'control_date.txt'
# Path to execution log file
logging_file_name = today_str + '_migracao_br.log'
# DSN for connection
odbc_dsn = 'DSN=<ODBC_DSN>'
# Path to Google Drive API json
google_api_key = '<PATH_TO_YOUR_GOOGLE_DRIVE_API_JSON>'
# Spreadsheet name
google_spreadsheet = '<GOOGLE_SPREADSHEET_NAME>'
#---------------------------- End - Setting variables ---------------------------#

#-------------------------- Start - Queries statements --------------------------#
# Returns max date loaded into discountrevenue table to check if there's new data
query_max_eventdate = """<SELECT_STATEMENT_TO_CHECK_LAST_DATE_AVAILABLE>"""
# Returns the number of discountids by year-month not exists in discountdetails table
query_check_discountdetails = """<SELECT_STATEMENT_TO_SOME_CHECK>"""
# Query to populate spreadsheet
query_main_data = """<SELECT_STATEMENT_TO_GET_MAIN_DATASET>"""
#--------------------------- End - Queries statements ---------------------------#

logging.basicConfig(filename=logging_file_name, level=logging.DEBUG, format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

logging.info('-----------------------------------------------------------')
logging.info('[INFO] Starting process')

ts = time.time()
start_time_str = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
start_time_dt = datetime.datetime.today()

# Try to connect to database using ODBC driver
try:
    con = pyodbc.connect(odbc_dsn)
except:
    error = sys.exc_info()
    logging.error('[ERROR] Connection failed')
    for e in error:
        logging.error('[ERROR] ' + str(e))
    ending_log(start_time_dt, True)
    exit()

# Try to execute query statement
try:
    cur = con.cursor()
    cur.execute(query_max_eventdate)
except:
    error = sys.exc_info()
    logging.error('[ERROR] Error getting max date from <source_table>')
    for e in error:
        logging.error('[ERROR] ' + str(e))
    ending_log(start_time_dt, True)
    exit()

# Fetch max date into local variable
fetch = cur.fetchall()
max_db_date = fetch[0][0]

# Try to open the file
if os.path.exists(execution_control_file_name):
    try:
        file = open(execution_control_file_name, 'r')
        last_date_ran = file.readline()
        last_date_ran_dt = datetime.datetime.strptime(last_date_ran, '%Y-%m-%d').date()
    except:
        logging.error('[ERROR] Error reading execution file control')
        ending_log(start_time_dt, True)
        exit()
else:
    last_date_ran_dt = datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date()

# Check if there's new data available to execute the main query
if max_db_date == last_date_ran_dt:
    logging.info('[INFO] Max date on <source_table> is equal than last date executed')
    ending_log(start_time_dt)
    exit()
else:
    # Try to execute query statement
    try:
        cur = con.cursor()
        cur.execute(query_check_discountdetails)
    except:
        error = sys.exc_info()
        logging.error('[ERROR] Error executing check query')
        for e in error:
            logging.error('[ERROR] ' + str(e))
        ending_log(start_time_dt, True)
        exit()
    check_results = cur.fetchall()
    for row in check_results:
        if row[1] > 10:  # Margin error
            logging.error('[ERROR] Error checking <some check> table!')
            # Message to identify some error, must change for each case and query return
            logging.error('[ERROR] <First Column> ' + row[0] + ' <xxxx> ' + str(row[1]) + ' <Second Column>')
            ending_log(start_time_dt, True)
            exit()

    try:
        cur.execute(query_main_data)
    except:
        error = sys.exc_info()
        logging.error('[ERROR] Error executing main query')
        for e in error:
            logging.error('[ERROR] ' + str(e))
        cur.close()
        con.close()
        ending_log(start_time_dt, True)
        exit()

    all_results = cur.fetchall()
    row_count = cur.rowcount
    cur.close()
    con.close()

    # use creds to create a client to interact with the Google Drive API
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(google_api_key, scope)
    client = gspread.authorize(creds)

    # Find a workbook by name and open the first sheet
    # Make sure you use the right name here.
    spreadsheet = client.open(google_spreadsheet)
    year = max_db_date.strftime('%Y')
    sheet = spreadsheet.worksheet('Base - ' + year) # In this case, worksheet's name = Base - 2018

    range = sheet.range('A1:H' + str(row_count)) # Set appropriate range

    i = 0

    for rows in all_results:
        for tuple in rows:
            range[i].value = tuple
            i = i + 1

    sheet.update_cells(range)

    file = open(execution_control_file_name, 'w')

    file.write(max_db_date.strftime('%Y-%m-%d'))
    file.close()

    ending_log(start_time_dt)